In [1]:
from docplex.mp.model import Model

# Dati di esempio
distances = [[0, 10, 20, 30], [10, 0, 15, 25], [20, 15, 0, 35], [30, 25, 35, 0]]
A = [(i, j) for i in range(len(distances)) for j in range(len(distances)) if i != j]
N0 = list(range(1, len(distances)))  # Nodi clienti (escluso il deposito)
v = 3  # Numero di veicoli
q = [0, 40, 29, 18]  # quantity of goods to collect/deliver
Q = 100  # Capacità massima del veicolo
L = 100  # Capacità massima della batteria
a = {(i, j): 2 for (i, j) in A}  # Consumo base per arco
b = {(i, j): 0.5 for (i, j) in A}  # Consumo dipendente dal carico
c_E = 1  # Costo per unità di energia consumata
c_F = 1  # Costo fisso per veicolo
c_M = 1  # Costo per unità di distanza
M = 100000  # big M

# Creazione del modello
mdl_energy = Model("EnergyOptimization")

# Variabili di decisione
x_energy = mdl_energy.binary_var_cube(len(distances), len(distances), v, name="x")  # Percorsi
f = mdl_energy.continuous_var_cube(
    len(distances), len(distances), v, name="f"
)  # Flussi di carico
w = mdl_energy.continuous_var_list(v, name="w")  # Consumo massimo di energia

# Funzione obiettivo: Minimizzare i costi energetici
mdl_energy.minimize(
    mdl_energy.sum(
        [
            mdl_energy.sum([c_F * x_energy[0, j, k] for j in range(1, len(distances))])
            + mdl_energy.sum([c_M * distances[i][j] for i, j in A])
            + c_E * w[k]
            for k in range(v)
        ]
    )
)

# Vincoli
for k in range(v):
    mdl_energy.add_constraint(mdl_energy.sum(x_energy[0, j, k] for j in N0) <= 1)

for i in N0:
    mdl_energy.add_constraint(
        mdl_energy.sum(
            x_energy[i, j, k] for k in range(v) for j in range(len(distances)) if j != i
        )
        == 1
    )

for i in range(len(distances)):
    for k in range(v):
        mdl_energy.add_constraint(
            mdl_energy.sum(x_energy[j, i, k] for j in range(len(distances)) if j != i)
            == mdl_energy.sum(x_energy[i, j, k] for j in range(len(distances)) if j != i)
        )

for i, j in A:
    for k in range(v):
        mdl_energy.add_constraint(q[j] * x_energy[i, j, k] <= f[i, j, k])
        mdl_energy.add_constraint(f[i, j, k] <= (Q - q[i]) * x_energy[i, j, k])

for i in N0:
    mdl_energy.add_constraint(
        mdl_energy.sum(f[j, i, k] for k in range(v) for j in range(len(distances)) if j != i)
        - mdl_energy.sum(f[i, j, k] for k in range(v) for j in range(len(distances)) if j != i)
        == q[i]
    )

for k in range(v):
    mdl_energy.add_constraint(
        mdl_energy.sum(a[i, j] * x_energy[i, j, k] + b[i, j] * f[i, j, k] for i, j in A)
        <= w[k]
    )

for k in range(v):
    mdl_energy.add_constraint(0 <= w[k])
    mdl_energy.add_constraint(w[k] <= L)

# Risoluzione del modello
solution = mdl_energy.solve(log_output=False)

if solution is None:
    print("Il modello non ha soluzione")
else:
    print("Costo totale:", solution.objective_value)
    for k in range(v):
        print(
            f"Consumo massimo del veicolo {k}: {solution.get_value(mdl_energy.get_var_by_name(f'w_{k}'))}"
        )

w = solution.get_values(w)

for w in w:
    if w == 0:
        print("\nEsistono uno o più veicoli che non vengono utilizzati")
        break

Costo totale: 868.5
Consumo massimo del veicolo 0: 13.0
Consumo massimo del veicolo 1: 18.5
Consumo massimo del veicolo 2: 24.0
